<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/ML_Most_used_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class Perceptron:
    def __init__(self, input_size, learning_rate=0.01, epochs=100):
        self.weights = np.zeros(input_size + 1)
        self.learning_rate = learning_rate
        self.epochs = epochs

    def activation_function(self, x):
        return 1 if x >= 0 else 0

    def predict(self, inputs):
        summation = np.dot(inputs, self.weights[1:]) + self.weights[0]
        return self.activation_function(summation)

    def train(self, training_inputs, labels):
        for _ in range(self.epochs):
            for inputs, label in zip(training_inputs, labels):
                prediction = self.predict(inputs)
                self.weights[1:] += self.learning_rate * (label - prediction) * inputs
                self.weights[0] += self.learning_rate * (label - prediction)


In [ ]:
import numpy as np

class FeedForwardNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.bias_input_hidden = np.zeros((1, self.hidden_size))

        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
        self.bias_hidden_output = np.zeros((1, self.output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, inputs):
        # Forward pass through the network
        self.hidden_sum = np.dot(inputs, self.weights_input_hidden) + self.bias_input_hidden
        self.hidden_output = self.sigmoid(self.hidden_sum)

        self.output_sum = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_hidden_output
        self.output = self.sigmoid(self.output_sum)

        return self.output

    def backward(self, inputs, targets, learning_rate):
        # Backpropagation
        # Calculate error
        output_error = targets - self.output
        output_delta = output_error * self.sigmoid_derivative(self.output)

        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)

        # Update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(output_delta) * learning_rate
        self.bias_hidden_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate

        self.weights_input_hidden += inputs.T.dot(hidden_delta) * learning_rate
        self.bias_input_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate

    def train(self, inputs, targets, learning_rate, epochs):
        for epoch in range(epochs):
            self.forward(inputs)
            self.backward(inputs, targets, learning_rate)
            if epoch % 100 == 0:
                loss = np.mean(np.square(targets - self.output))
                print(f'Epoch {epoch}: Loss = {loss}')


In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

class RadialBasisNetwork:
    def __init__(self, input_size, hidden_size, output_size, sigma=1.0, learning_rate=0.1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.sigma = sigma
        self.learning_rate = learning_rate

        # Initialize random weights for hidden layer
        self.centers = np.random.rand(hidden_size, input_size)
        self.weights = np.random.rand(output_size, hidden_size)
        self.bias = np.random.rand(output_size)

    def _gaussian(self, x, c):
        return np.exp(-np.linalg.norm(x-c)**2 / (2 * self.sigma ** 2))

    def _forward(self, X):
        # Compute activations for hidden layer
        G = np.array([self._gaussian(X, c) for c in self.centers])

        # Compute output of the network
        output = np.dot(self.weights, G) + self.bias
        return output, G

    def _backward(self, X, y, output, G):
        # Compute gradients for weights and bias
        d_weights = -np.outer(y - output, G)
        d_bias = -(y - output)

        # Update weights and bias
        self.weights -= self.learning_rate * d_weights
        self.bias -= self.learning_rate * d_bias

    def train(self, X, y, epochs=100):
        for _ in range(epochs):
            for i in range(len(X)):
                output, G = self._forward(X[i])
                self._backward(X[i], y[i], output, G)

    def predict(self, X):
        output = []
        for i in range(len(X)):
            prediction, _ = self._forward(X[i])
            output.append(prediction)
        return np.array(output)

# Example usage
# Assuming X_train and y_train are your training data
X_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_train = np.array([0, 1, 1, 0])

# Initialize and train the RBF network
rbf_network = RadialBasisNetwork(input_size=2, hidden_size=4, output_size=1)
rbf_network.train(X_train, y_train)

# Make predictions
X_test = np.array([[0.5, 0.5], [0.2, 0.8]])
predictions = rbf_network.predict(X_test)
print(predictions)


[[0.49765259]
 [0.55978663]]


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Define the model
def create_model(input_dim, hidden_layers, output_dim):
    model = Sequential()

    # Add input layer
    model.add(Dense(hidden_layers[0], input_dim=input_dim, activation='relu'))

    # Add hidden layers
    for i in range(1, len(hidden_layers)):
        model.add(Dense(hidden_layers[i], activation='relu'))

    # Add output layer
    model.add(Dense(output_dim, activation='softmax'))

    return model

# Example usage
input_dim = 10  # Dimension of input data
hidden_layers = [64, 32]  # List of hidden layer sizes
output_dim = 2  # Dimension of output (e.g., number of classes)
model = create_model(input_dim, hidden_layers, output_dim)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2850 (11.13 KB)
Trainable params: 2850 (11.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define the sequence length, number of features, and number of units in the RNN layer
sequence_length = 10
num_features = 1
num_units = 32

# Generate some dummy data
X = np.random.randn(1000, sequence_length, num_features)
y = np.random.randint(0, 2, size=(1000,))

# Build the RNN model
model = Sequential([
    SimpleRNN(num_units, input_shape=(sequence_length, num_features)),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(X)


Epoch 1/10
32/32 [==============================] - 2s 4ms/step - loss: 0.6953 - accuracy: 0.5340
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6879 - accuracy: 0.5470
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6856 - accuracy: 0.5560
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6844 - accuracy: 0.5590
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6843 - accuracy: 0.5620
Epoch 6/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6833 - accuracy: 0.5740
Epoch 7/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6815 - accuracy: 0.5650
Epoch 8/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6813 - accuracy: 0.5590
Epoch 9/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6806 - accuracy: 0.5630
Epoch 10/10
32/32 [==============================] - 0s 2ms/step


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Generate some sample data
# Replace this with your actual data
# Example: Random data with 100 samples, 10 timesteps, and 20 features
num_samples = 100
timesteps = 10
features = 20
X_train = np.random.rand(num_samples, timesteps, features)
y_train = np.random.randint(0, 2, size=(num_samples,))

# Define the LSTM model
def build_lstm_model(input_shape, num_units, num_classes):
    model = Sequential([
        LSTM(units=num_units, input_shape=input_shape),
        Dense(units=num_classes, activation='softmax')
    ])
    return model

# Example usage:
num_units = 64  # Number of LSTM units
num_classes = 2  # Number of output classes

# Build the LSTM model
lstm_model = build_lstm_model(input_shape=(timesteps, features), num_units=num_units, num_classes=num_classes)

# Compile the model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
# Replace X_test and y_test with your test data
# loss, accuracy = lstm_model.evaluate(X_test, y_test)
# print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/10
3/3 [==============================] - 3s 336ms/step - loss: 0.7155 - accuracy: 0.4250 - val_loss: 0.6791 - val_accuracy: 0.6000
Epoch 2/10
3/3 [==============================] - 0s 38ms/step - loss: 0.6939 - accuracy: 0.5750 - val_loss: 0.6868 - val_accuracy: 0.6000
Epoch 3/10
3/3 [==============================] - 0s 24ms/step - loss: 0.6992 - accuracy: 0.5750 - val_loss: 0.6811 - val_accuracy: 0.6000
Epoch 4/10
3/3 [==============================] - 0s 24ms/step - loss: 0.6905 - accuracy: 0.5750 - val_loss: 0.6768 - val_accuracy: 0.6000
Epoch 5/10
3/3 [==============================] - 0s 24ms/step - loss: 0.6812 - accuracy: 0.5750 - val_loss: 0.6777 - val_accuracy: 0.6000
Epoch 6/10
3/3 [==============================] - 0s 32ms/step - loss: 0.6770 - accuracy: 0.5750 - val_loss: 0.6799 - val_accuracy: 0.6000
Epoch 7/10
3/3 [==============================] - 0s 24ms/step - loss: 0.6781 - accuracy: 0.5750 - val_loss: 0.6826 - val_accuracy: 0.6000
Epoch 8/10
3/3 [==========

In [ ]:
import tensorflow as tf

class GRU(tf.keras.Model):
    def __init__(self, units, input_dim):
        super(GRU, self).__init__()
        self.units = units
        self.gru_cell = tf.keras.layers.GRUCell(units)
        self.dense = tf.keras.layers.Dense(input_dim)

    def call(self, inputs):
        # Initialize hidden state
        hidden_state = tf.zeros([inputs.shape[0], self.units])

        # Iterate over each time step
        for t in range(inputs.shape[1]):
            # Update hidden state using GRU cell
            hidden_state = self.gru_cell(inputs[:, t, :], [hidden_state])[0]

        # Pass the final hidden state through a dense layer
        output = self.dense(hidden_state)
        return output


In [ ]:
import tensorflow as tf
import numpy as np

class Autoencoder:
    def __init__(self, input_dim, encoding_dim):
        self.input_dim = input_dim
        self.encoding_dim = encoding_dim

        # Define placeholders for input and target
        self.input_data = tf.placeholder(tf.float32, [None, input_dim])
        self.target_data = tf.placeholder(tf.float32, [None, input_dim])

        # Define weights and biases for encoding and decoding layers
        self.weights = {
            'encoder': tf.Variable(tf.random_normal([input_dim, encoding_dim])),
            'decoder': tf.Variable(tf.random_normal([encoding_dim, input_dim]))
        }
        self.biases = {
            'encoder': tf.Variable(tf.random_normal([encoding_dim])),
            'decoder': tf.Variable(tf.random_normal([input_dim]))
        }

        # Define encoder and decoder operations
        self.encoded = tf.nn.sigmoid(tf.add(tf.matmul(self.input_data, self.weights['encoder']), self.biases['encoder']))
        self.decoded = tf.nn.sigmoid(tf.add(tf.matmul(self.encoded, self.weights['decoder']), self.biases['decoder']))

        # Define loss function and optimizer
        self.loss = tf.reduce_mean(tf.square(self.decoded - self.target_data))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(self.loss)

    def train(self, input_data, target_data, epochs=100, batch_size=128):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            num_batches = input_data.shape[0] // batch_size

            for epoch in range(epochs):
                avg_loss = 0.0
                for i in range(num_batches):
                    batch_input = input_data[i * batch_size: (i + 1) * batch_size]
                    batch_target = target_data[i * batch_size: (i + 1) * batch_size]
                    _, loss = sess.run([self.optimizer, self.loss], feed_dict={self.input_data: batch_input, self.target_data: batch_target})
                    avg_loss += loss / num_batches
                print("Epoch:", '%04d' % (epoch + 1), "loss=", "{:.9f}".format(avg_loss))

            self.weights_enc, self.biases_enc, self.weights_dec, self.biases_dec = sess.run([self.weights['encoder'], self.biases['encoder'], self.weights['decoder'], self.biases['decoder']])


In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers
# this sampling layer is the bottleneck layer of variational autoencoder,
# it uses the output from two dense layers z_mean and z_log_var as input,
# convert them into normal distribution and pass them to the decoder layer

class Sampling(layers.Layer):
	"""Uses (mean, log_var) to sample z, the vector encoding a digit."""

	def call(self, inputs):
		mean, log_var = inputs
		batch = tf.shape(mean)[0]
		dim = tf.shape(mean)[1]
		epsilon = tf.random.normal(shape=(batch, dim))
		return mean + tf.exp(0.5 * log_var) * epsilon
latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(128, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
mean = layers.Dense(latent_dim, name="mean")(x)
log_var = layers.Dense(latent_dim, name="log_var")(x)
z = Sampling()([mean, log_var])
encoder = keras.Model(encoder_inputs, [mean, log_var, z], name="encoder")
encoder.summary()
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(128, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 14, 14, 64)           640       ['input_8[0][0]']             
                                                                                                  
 conv2d_3 (Conv2D)           (None, 7, 7, 128)            73856     ['conv2d_2[0][0]']            
                                                                                                  
 flatten_4 (Flatten)         (None, 6272)                 0         ['conv2d_3[0][0]']            
                                                                                            

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

class DenoisingAutoencoder:
    def __init__(self, input_dim, hidden_dim, noise_factor=0.2):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.noise_factor = noise_factor

        # Build the model
        self._build_model()

    def _build_model(self):
        # Define input layer
        input_layer = Input(shape=(self.input_dim,))

        # Add noise to the input
        noisy_input = GaussianNoise(self.noise_factor)(input_layer)

        # Define encoder layer
        encoder_layer = Dense(self.hidden_dim, activation='relu')(noisy_input)

        # Define decoder layer
        decoder_layer = Dense(self.input_dim, activation='sigmoid')(encoder_layer)

        # Create the autoencoder model
        self.autoencoder = Model(inputs=input_layer, outputs=decoder_layer)

        # Create the encoder model
        self.encoder = Model(inputs=input_layer, outputs=encoder_layer)

        # Compile the autoencoder model
        self.autoencoder.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')

    def train(self, X_train, epochs=10, batch_size=32):
        self.autoencoder.fit(X_train, X_train,
                             epochs=epochs,
                             batch_size=batch_size,
                             shuffle=True)

    def encode(self, X):
        return self.encoder.predict(X)

    def decode(self, encoded_X):
        return self.autoencoder.predict(encoded_X)


In [ ]:
import tensorflow as tf
import numpy as np

class StackedAutoencoder:
    def __init__(self, layers, learning_rate=0.01, num_epochs=100):
        self.layers = layers
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = []
        self.biases = []

        # Define placeholders for input data
        self.input_data = tf.placeholder(tf.float32, [None, self.layers[0]])

        # Initialize weights and biases for each layer
        for i in range(1, len(self.layers)):
            W = tf.Variable(tf.random_normal([self.layers[i-1], self.layers[i]]))
            b = tf.Variable(tf.random_normal([self.layers[i]]))
            self.weights.append(W)
            self.biases.append(b)

        # Build the encoder
        self.encoder = self._build_encoder()

        # Build the decoder
        self.decoder = self._build_decoder()

        # Define the loss function and optimizer
        self.loss = tf.reduce_mean(tf.square(self.input_data - self.decoder))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)

        # Initialize TensorFlow session
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def _build_encoder(self):
        layer = self.input_data
        for i in range(len(self.weights)):
            layer = tf.nn.sigmoid(tf.add(tf.matmul(layer, self.weights[i]), self.biases[i]))
        return layer

    def _build_decoder(self):
        layer = self.encoder
        for i in range(len(self.weights)-1, -1, -1):
            layer = tf.nn.sigmoid(tf.add(tf.matmul(layer, tf.transpose(self.weights[i])), self.biases[i]))
        return layer

    def train(self, data):
        for epoch in range(self.num_epochs):
            _, loss = self.sess.run([self.optimizer, self.loss], feed_dict={self.input_data: data})
            print("Epoch:", epoch+1, "Loss:", loss)

    def encode(self, data):
        return self.sess.run(self.encoder, feed_dict={self.input_data: data})

    def decode(self, encoded_data):
        return self.sess.run(self.decoder, feed_dict={self.encoder: encoded_data})


In [ ]:
import random

def estimate_pi(num_samples):
    inside_circle = 0
    total_samples = num_samples

    for _ in range(num_samples):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            inside_circle += 1

    estimated_pi = 4 * inside_circle / total_samples
    return estimated_pi

num_samples = 10000
estimated_pi = estimate_pi(num_samples)
print(f"Estimated value of π with {num_samples} samples: {estimated_pi}")


Estimated value of π with 10000 samples: 3.1324


In [ ]:
import numpy as np

class BoltzmannMachine:
    def __init__(self, num_visible, num_hidden):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.weights = np.random.normal(0, 0.1, (num_visible, num_hidden))
        self.visible_bias = np.zeros(num_visible)
        self.hidden_bias = np.zeros(num_hidden)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def energy(self, visible_states, hidden_states):
        visible_term = -np.dot(visible_states, self.visible_bias)
        hidden_term = -np.dot(hidden_states, self.hidden_bias)
        interaction_term = -np.dot(np.dot(visible_states, self.weights), hidden_states.T)
        return visible_term + hidden_term + interaction_term

    def sample_hidden_given_visible(self, visible_states):
        hidden_activations = np.dot(visible_states, self.weights) + self.hidden_bias
        hidden_probs = self.sigmoid(hidden_activations)
        return (np.random.uniform(size=hidden_probs.shape) < hidden_probs).astype(int)

    def sample_visible_given_hidden(self, hidden_states):
        visible_activations = np.dot(hidden_states, self.weights.T) + self.visible_bias
        visible_probs = self.sigmoid(visible_activations)
        return (np.random.uniform(size=visible_probs.shape) < visible_probs).astype(int)

    def gibbs_sampling(self, initial_visible, steps=1):
        visible_states = initial_visible.copy()
        for _ in range(steps):
            hidden_states = self.sample_hidden_given_visible(visible_states)
            visible_states = self.sample_visible_given_hidden(hidden_states)
        return visible_states

    def train(self, data, learning_rate=0.1, num_epochs=100, k=1):
        num_data = data.shape[0]
        for epoch in range(num_epochs):
            np.random.shuffle(data)
            for sample in data:
                visible_states = sample.copy()
                for _ in range(k):
                    hidden_states = self.sample_hidden_given_visible(visible_states)
                    visible_states = self.sample_visible_given_hidden(hidden_states)
                positive_grad = np.outer(sample, self.sigmoid(np.dot(visible_states, self.weights) + self.hidden_bias))
                negative_grad = np.outer(visible_states, self.sigmoid(np.dot(hidden_states, self.weights) + self.visible_bias))  # Corrected this line
                self.weights += learning_rate * (positive_grad - negative_grad)
                self.visible_bias += learning_rate * (sample - visible_states)
                self.hidden_bias += learning_rate * (np.mean(sample, axis=0) - np.mean(hidden_states, axis=0))

# Example usage:
# Initialize Boltzmann Machine
bm = BoltzmannMachine(num_visible=4, num_hidden=2)
# Generate some training data
training_data = np.array([[0, 1, 1, 0], [1, 0, 1, 1], [0, 1, 0, 1]])
# Train the Boltzmann Machine
bm.train(training_data)
# Sample from the Boltzmann Machine
visible_sample = np.array([0, 1, 0, 0])
sampled_visible = bm.gibbs_sampling(visible_sample)
print("Sampled Visible:", sampled_visible)


In [ ]:
import numpy as np

class RBM:
    def __init__(self, num_visible, num_hidden):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.weights = np.random.normal(0, 0.1, (num_visible, num_hidden))
        self.visible_bias = np.zeros(num_visible)
        self.hidden_bias = np.zeros(num_hidden)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def gibbs_sampling(self, visible_probs):
        hidden_probs = self.sigmoid(np.dot(visible_probs, self.weights) + self.hidden_bias)
        hidden_states = np.random.binomial(1, hidden_probs)
        visible_recon_probs = self.sigmoid(np.dot(hidden_states, self.weights.T) + self.visible_bias)
        return hidden_probs, hidden_states, visible_recon_probs

    def train(self, data, learning_rate=0.1, epochs=10, batch_size=10):
        num_samples = data.shape[0]
        for epoch in range(epochs):
            np.random.shuffle(data)
            for i in range(0, num_samples, batch_size):
                batch_data = data[i:i+batch_size]
                visible_probs = batch_data
                hidden_probs, hidden_states, visible_recon_probs = self.gibbs_sampling(visible_probs)

                positive_associations = np.dot(visible_probs.T, hidden_probs)
                negative_associations = np.dot(visible_recon_probs.T, hidden_states)

                self.weights += learning_rate * ((positive_associations - negative_associations) / batch_size)
                self.visible_bias += learning_rate * np.mean(visible_probs - visible_recon_probs, axis=0)
                self.hidden_bias += learning_rate * np.mean(hidden_probs - hidden_states, axis=0)

    def sample_hidden_given_visible(self, visible_probs):
        return self.sigmoid(np.dot(visible_probs, self.weights) + self.hidden_bias)

    def sample_visible_given_hidden(self, hidden_states):
        return self.sigmoid(np.dot(hidden_states, self.weights.T) + self.visible_bias)

    def gibbs_sampling_k_steps(self, visible_probs, k=1):
        for _ in range(k):
            hidden_probs = self.sample_hidden_given_visible(visible_probs)
            hidden_states = np.random.binomial(1, hidden_probs)
            visible_probs = self.sample_visible_given_hidden(hidden_states)
        return visible_probs

# Example usage:
# Assume we have binary data where each row represents a sample
# and each column represents a feature.
data = np.array([[0, 1, 1, 0], [1, 0, 0, 1], [1, 1, 0, 0], [0, 0, 1, 1]])

# Define RBM parameters
num_visible = data.shape[1]
num_hidden = 100

# Create RBM object
rbm = RBM(num_visible, num_hidden)

# Train RBM
rbm.train(data, learning_rate=0.1, epochs=10, batch_size=10)

# Generate samples using Gibbs sampling
visible_probs = np.array([[0, 1, 1, 0]])
reconstructed_visible_probs = rbm.gibbs_sampling_k_steps(visible_probs, k=100)
print("Original data:", visible_probs)
print("Reconstructed data:", reconstructed_visible_probs)


Original data: [[0 1 1 0]]
Reconstructed data: [[0.4935733  0.63074654 0.39849566 0.51239904]]


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

class DBN(models.Sequential):
    def __init__(self, hidden_units):
        super(DBN, self).__init__()
        self.hidden_layers = []
        for units in hidden_units:
            self.add(layers.Dense(units, activation='sigmoid', kernel_initializer='glorot_normal'))
            self.add(layers.Dropout(0.2))  # Optional dropout layer for regularization
            self.hidden_layers.append(units)

    def pretrain(self, train_data, learning_rate=0.01, epochs=50, batch_size=128):
        for i in range(len(self.hidden_layers) - 1):
            rbm = RBM(visible_units=self.hidden_layers[i], hidden_units=self.hidden_layers[i + 1])
            rbm.train(train_data, learning_rate=learning_rate, epochs=epochs, batch_size=batch_size)
            train_data = rbm.transform(train_data)

    def fine_tune(self, train_data, train_labels, learning_rate=0.01, epochs=50, batch_size=128):
        self.add(layers.Dense(train_labels.shape[1], activation='softmax', kernel_initializer='glorot_normal'))
        self.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                     loss='categorical_crossentropy', metrics=['accuracy'])
        self.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size)

class RBM(tf.keras.Model):
    def __init__(self, visible_units, hidden_units):
        super(RBM, self).__init__()
        self.weights = tf.Variable(tf.random.normal([visible_units, hidden_units], stddev=0.1))
        self.visible_bias = tf.Variable(tf.zeros([visible_units]))
        self.hidden_bias = tf.Variable(tf.zeros([hidden_units]))

    def sample_hidden(self, visible_probabilities):
        hidden_probabilities = tf.nn.sigmoid(tf.matmul(visible_probabilities, self.weights) + self.hidden_bias)
        return tf.nn.relu(tf.sign(hidden_probabilities - tf.random.uniform(tf.shape(hidden_probabilities))))

    def sample_visible(self, hidden_probabilities):
        visible_probabilities = tf.nn.sigmoid(tf.matmul(hidden_probabilities, tf.transpose(self.weights)) + self.visible_bias)
        return tf.nn.relu(tf.sign(visible_probabilities - tf.random.uniform(tf.shape(visible_probabilities))))

    def contrastive_divergence(self, input_data, k=1):
        positive_hidden_probabilities = self.sample_hidden(input_data)
        for i in range(k):
            positive_hidden_states = tf.nn.relu(tf.sign(positive_hidden_probabilities - tf.random.uniform(tf.shape(positive_hidden_probabilities))))
            negative_visible_probabilities = self.sample_visible(positive_hidden_states)
            negative_hidden_probabilities = self.sample_hidden(negative_visible_probabilities)
        return input_data, positive_hidden_probabilities, negative_visible_probabilities, negative_hidden_probabilities

    def train(self, input_data, learning_rate=0.01, epochs=50, batch_size=128, k=1):
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        for epoch in range(epochs):
            for i in range(0, input_data.shape[0], batch_size):
                batch_data = input_data[i:i+batch_size]
                with tf.GradientTape() as tape:
                    positive_visible_probabilities, positive_hidden_probabilities, negative_visible_probabilities, negative_hidden_probabilities = self.contrastive_divergence(batch_data, k)
                    loss = tf.reduce_mean(batch_data - negative_visible_probabilities)
                gradients = tape.gradient(loss, self.trainable_variables)
                optimizer.apply_gradients(zip(gradients, self.trainable_variables))
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')

# Example usage:
# 1. Prepare your dataset
# 2. Instantiate a DBN object
# 3. Pretrain the DBN using pretrain method
# 4. Fine-tune the DBN using fine_tune method

# Note: This is a basic implementation. For real-world usage, you may need to customize it based on your specific requirements.


In [1]:
import openai

# Set your OpenAI API key
api_key = "your-api-key"
openai.api_key = api_key

# Set the prompt for text generation
prompt_text = "Once upon a time,"

# Generate text using the completion endpoint
response = openai.Completion.create(
  engine="text-davinci-002",  # You can choose the GPT model here
  prompt=prompt_text,
  max_tokens=50  # Maximum number of tokens to generate
)

# Get the generated text from the response
generated_text = response.choices[0].text.strip()

# Print the generated text
print("Generated Text:", generated_text)


ModuleNotFoundError: No module named 'openai'

In [ ]:
import tensorflow as tf

# Define the DCN model
def create_dcn_model():
    model = tf.keras.Sequential([
        # Convolutional layers
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        # Flatten layer
        tf.keras.layers.Flatten(),
        # Dense layers
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')  # 10 classes for MNIST dataset
    ])
    return model

# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channel dimension to the images (required for Conv2D)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Create the DCN model
model = create_dcn_model()

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)


Epoch 1/5
1875/1875 [==============================] - 56s 29ms/step - loss: 0.1519 - accuracy: 0.9535
Epoch 2/5
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0475 - accuracy: 0.9854
Epoch 3/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0328 - accuracy: 0.9897
Epoch 4/5
1875/1875 [==============================] - 57s 31ms/step - loss: 0.0263 - accuracy: 0.9918
Epoch 5/5
313/313 [==============================] - 3s 9ms/step - loss: 0.0307 - accuracy: 0.9897
Test accuracy: 0.9897000193595886


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

class DomainAdaptiveCNN(nn.Module):
    def __init__(self, num_classes):
        super(DomainAdaptiveCNN, self).__init__()
        self.feature_extractor = models.resnet18(pretrained=True)
        self.feature_extractor.fc = nn.Identity()  # Remove the classification layer

        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        outputs = self.classifier(features)
        return outputs

# Example usage
source_data_loader = ...
target_data_loader = ...
num_classes = ...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the DCN model
model = DomainAdaptiveCNN(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for source_images, source_labels in source_data_loader:
        source_images, source_labels = source_images.to(device), source_labels.to(device)

        optimizer.zero_grad()
        source_outputs = model(source_images)
        source_loss = criterion(source_outputs, source_labels)
        source_loss.backward()
        optimizer.step()

    # Adapt the model to the target domain
    for target_images, _ in target_data_loader:
        target_images = target_images.to(device)
        target_outputs = model(target_images)
        target_loss = -torch.mean(target_outputs)  # Maximizing the discrepancy between source and target
        target_loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Source Loss: {source_loss.item():.4f}, Target Loss: {target_loss.item():.4f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define DCIGN architecture
latent_dim = 100
input_shape = (28, 28, 1)  # Example input shape for MNIST images

# Encoder
inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
x = LeakyReLU(alpha=0.2)(x)
x = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)
x = LeakyReLU(alpha=0.2)(x)
x = Flatten()(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = tf.keras.layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
x = Dense(7 * 7 * 64, activation='relu')(decoder_inputs)
x = Reshape((7, 7, 64))(x)
x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
x = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
outputs = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

# Define models
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
decoder = Model(decoder_inputs, outputs, name='decoder')

# Define the full DCIGN model
outputs = decoder(encoder(inputs)[2])
dcign = Model(inputs, outputs, name='dcign')

# Define loss function
reconstruction_loss = tf.keras.losses.mean_squared_error(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
reconstruction_loss *= input_shape[0] * input_shape[1]
kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
dcign.add_loss(vae_loss)

# Compile the model
dcign.compile(optimizer='adam')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Define the generator network
def build_generator(latent_dim):
    generator = Sequential()
    generator.add(Dense(128, input_dim=latent_dim, activation='relu'))
    generator.add(Dense(784, activation='sigmoid'))
    generator.add(Reshape((28, 28)))
    return generator

# Define the discriminator network
def build_discriminator(input_shape):
    discriminator = Sequential()
    discriminator.add(Flatten(input_shape=input_shape))
    discriminator.add(Dense(128, activation='relu'))
    discriminator.add(Dense(1, activation='sigmoid'))
    return discriminator

# Define the combined GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan = Sequential()
    gan.add(generator)
    gan.add(discriminator)
    gan.compile(loss='binary_crossentropy', optimizer=Adam())
    return gan

# Load the MNIST dataset
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 255.0
X_train = X_train.reshape((-1, 28, 28))

# Define hyperparameters
latent_dim = 100
epochs = 100
batch_size = 32

# Build the generator and discriminator models
generator = build_generator(latent_dim)
discriminator = build_discriminator(X_train[0].shape)

# Compile the discriminator model
discriminator.compile(loss='binary_crossentropy', optimizer=Adam())

# Build the GAN model
gan = build_gan(generator, discriminator)

# Training loop
for epoch in range(epochs):
    for batch in range(X_train.shape[0] // batch_size):
        # Train the discriminator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_images = generator.predict(noise)
        real_images = X_train[np.random.randint(0, X_train.shape[0], batch_size)]
        X = np.concatenate([real_images, fake_images])
        y = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        discriminator_loss = discriminator.train_on_batch(X, y)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        y = np.ones((batch_size, 1))
        generator_loss = gan.train_on_batch(noise, y)

    # Print progress
    print(f'Epoch: {epoch + 1}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}')

# Generate some images with the trained generator
noise = np.random.normal(0, 1, (10, latent_dim))
generated_images = generator.predict(noise)

# Plot generated images
plt.figure(figsize=(10, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(generated_images[i], cmap='gray')
    plt.axis('off')
plt.tight_layout()
plt.show()


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 27ms/step
Epoch: 11, Discriminator Loss: 0.2943277359008789, Generator Loss: 1.6275660991668701
1/1 [==============================] - 0s 26ms/step
Epoch: 12, Discriminator Loss: 0.3582492470741272, Generator Loss: 1.4400594234466553
1/1 [==============================] - 0s 26ms/step
Epoch: 13, Discriminator Loss: 0.5460714101791382, Generator Loss: 2.027479648590088
1/1 [==============================] - 0s 23ms/step


In [ ]:
!pip install openai

import openai

# Set your OpenAI API key
api_key = "your-api-key"
openai.api_key = api_key

# Set the prompt for text generation
prompt_text = "Once upon a time,"

# Generate text using the completion endpoint
response = openai.Completion.create(
  engine="text-davinci-002",  # You can choose the GPT model here
  prompt=prompt_text,
  max_tokens=50  # Maximum number of tokens to generate
)

# Get the generated text from the response
generated_text = response.choices[0].text.strip()

# Print the generated text
print("Generated Text:", generated_text)


In [4]:
import numpy as np

class LiquidStateMachine:
    def __init__(self, input_size, reservoir_size, connectivity=0.1, spectral_radius=0.9):
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.connectivity = connectivity
        self.spectral_radius = spectral_radius
        self.W_in = np.random.uniform(-1, 1, size=(reservoir_size, input_size))
        self.W_res = np.random.rand(reservoir_size, reservoir_size)
        self.W_res[np.random.rand(reservoir_size, reservoir_size) > connectivity] = 0
        self.W_res *= spectral_radius / max(np.linalg.eigvals(self.W_res)).real

    def train(self, input_data, output_data):
        reservoir_states = np.zeros((len(input_data), self.reservoir_size))
        for t in range(1, len(input_data)):
            reservoir_states[t] = np.tanh(np.dot(self.W_in, input_data[t]) + np.dot(self.W_res, reservoir_states[t - 1]))

        self.W_out = np.linalg.lstsq(reservoir_states, output_data, rcond=None)[0]

    def predict(self, input_data):
        reservoir_states = np.zeros((len(input_data), self.reservoir_size))
        for t in range(1, len(input_data)):
            reservoir_states[t] = np.tanh(np.dot(self.W_in, input_data[t]) + np.dot(self.W_res, reservoir_states[t - 1]))

        return np.dot(reservoir_states, self.W_out)


# Example usage:
input_size = 1
reservoir_size = 100
connectivity = 0.1
spectral_radius = 0.9

# Create an instance of Liquid State Machine
lsm = LiquidStateMachine(input_size, reservoir_size, connectivity, spectral_radius)

# Generate some training data
input_data = np.random.rand(100, input_size)
output_data = np.sin(input_data)

# Train the LSM
lsm.train(input_data, output_data)

# Generate some test data
test_input_data = np.random.rand(50, input_size)

# Make predictions
predictions = lsm.predict(test_input_data)

print("Predictions:", predictions)


Predictions: [[0.        ]
 [0.76920944]
 [0.77331359]
 [0.73364773]
 [0.35788958]
 [0.35005558]
 [0.28708629]
 [0.69679467]
 [0.83060682]
 [0.44118704]
 [0.82904272]
 [0.82943437]
 [0.74204046]
 [0.7006598 ]
 [0.15345734]
 [0.71661664]
 [0.02868252]
 [0.29545869]
 [0.68447907]
 [0.41357579]
 [0.38641312]
 [0.10392849]
 [0.74789284]
 [0.00453959]
 [0.01990853]
 [0.49604718]
 [0.39922565]
 [0.59932141]
 [0.82824044]
 [0.65642206]
 [0.77576941]
 [0.30443826]
 [0.44194821]
 [0.64435972]
 [0.75142863]
 [0.06100826]
 [0.78833154]
 [0.76734487]
 [0.53155587]
 [0.07831329]
 [0.24837469]
 [0.67795671]
 [0.78069053]
 [0.78891648]
 [0.77830153]
 [0.4492344 ]
 [0.23388174]
 [0.30807504]
 [0.11228929]
 [0.57835101]]


In [5]:
import numpy as np

class ELM:
    def __init__(self, n_hidden):
        self.n_hidden = n_hidden
        self.input_weights = None
        self.output_weights = None

    def _add_bias(self, X):
        return np.hstack([X, np.ones((X.shape[0], 1))])

    def _relu(self, X):
        return np.maximum(X, 0)

    def fit(self, X, y):
        X = self._add_bias(X)
        self.input_weights = np.random.randn(X.shape[1], self.n_hidden)
        H = self._relu(X.dot(self.input_weights))
        self.output_weights = np.linalg.pinv(H).dot(y)

    def predict(self, X):
        X = self._add_bias(X)
        H = self._relu(X.dot(self.input_weights))
        return H.dot(self.output_weights)

# Example usage:
# Generate some synthetic data
X_train = np.random.randn(100, 10)
y_train = np.random.randint(0, 2, 100)

X_test = np.random.randn(10, 10)

# Create and train the ELM model
elm = ELM(n_hidden=50)
elm.fit(X_train, y_train)

# Make predictions
predictions = elm.predict(X_test)
print(predictions)


[ 1.70252904  0.70282826  0.51420945  0.22156233  0.56034777  1.34141465
  1.95843512 -0.09210465  0.0700191   0.37503361]


In [6]:
import numpy as np

class EchoStateNetwork:
    def __init__(self, input_size, reservoir_size, output_size, spectral_radius=0.9, sparsity=0.1):
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity

        # Initialize reservoir weights
        self.W_in = np.random.rand(reservoir_size, input_size) - 0.5
        self.W_res = np.random.rand(reservoir_size, reservoir_size) - 0.5
        self.W_out = None

        # Scale the reservoir weights
        rho_W_res = np.max(np.abs(np.linalg.eigvals(self.W_res)))
        self.W_res *= spectral_radius / rho_W_res

        # Sparsify the reservoir weights
        mask = np.random.rand(reservoir_size, reservoir_size) < sparsity
        self.W_res *= mask

    def train(self, inputs, targets, washout=100):
        # Initialize reservoir states
        X = np.zeros((self.reservoir_size, inputs.shape[0]))

        # Run the reservoir with input data
        for t in range(inputs.shape[0]):
            if t < washout:
                u = inputs[t, :]
                x_in = np.dot(self.W_in, u)
                x_res = np.dot(self.W_res, X[:, t-1])
                X[:, t] = np.tanh(x_in + x_res)
            else:
                break

        # Train the output weights
        X_T = X[:, washout:].T
        self.W_out = np.dot(np.linalg.pinv(X_T), targets[washout:])

    def predict(self, inputs):
        outputs = np.zeros((inputs.shape[0], self.output_size))

        # Run the reservoir with input data
        X = np.zeros((self.reservoir_size, inputs.shape[0]))
        for t in range(inputs.shape[0]):
            u = inputs[t, :]
            x_in = np.dot(self.W_in, u)
            x_res = np.dot(self.W_res, X[:, t-1])
            X[:, t] = np.tanh(x_in + x_res)

            # Compute output
            outputs[t, :] = np.dot(self.W_out.T, X[:, t])

        return outputs


In [7]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def res_block(input_tensor, filters, kernel_size, strides=(1, 1), activation='relu'):
    """Residual block"""
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same')(input_tensor)
    shortcut = BatchNormalization()(shortcut)
    x = Add()([x, shortcut])
    x = Activation(activation)(x)
    return x

def build_resnet(input_shape, num_classes):
    """Build ResNet model"""
    input_tensor = Input(shape=input_shape)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = res_block(x, 64, (3, 3))
    x = res_block(x, 64, (3, 3))
    x = res_block(x, 64, (3, 3))
    x = AveragePooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(input_tensor, x)
    return model

# Define input shape and number of classes
input_shape = (32, 32, 3)  # Example input shape for CIFAR-10
num_classes = 10

# Build ResNet model
model = build_resnet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 64)           9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 16, 16, 64)           256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 16, 16, 64)           0         ['batch_normalization[0][0

In [ ]:
class DNC(tf.keras.Model):
    def __init__(self, memory_size, word_size, num_read_heads, num_write_heads):
        super(DNC, self).__init__()
        self.memory_size = memory_size
        self.word_size = word_size
        self.num_read_heads = num_read_heads
        self.num_write_heads = num_write_heads

        # Define controller (e.g., LSTM)
        self.controller = LSTM(units=128, return_sequences=True, return_state=True)

        # Define read and write heads
        self.read_head = Dense(self.word_size * self.num_read_heads, activation='sigmoid')
        self.write_head = Dense(self.word_size * self.num_write_heads, activation='sigmoid')

        # Initialize memory matrix
        self.memory_matrix = tf.Variable(tf.random.normal(shape=(1, self.memory_size, self.word_size)))

    def call(self, inputs):
        # Run controller
        controller_outputs, state_h, state_c = self.controller(inputs)

        # Read from memory
        read_vector = self.read_head(controller_outputs)

        # Write to memory
        write_vector = self.write_head(controller_outputs)
        write_vector = tf.reshape(write_vector, shape=(-1, 1, self.word_size, self.num_write_heads))  # Reshape to match desired shape
        write_address = tf.random.uniform(shape=(1, 1, self.memory_size), minval=0, maxval=1, dtype=tf.float32)
        self.memory_matrix = self.memory_matrix * (1 - write_address) + tf.matmul(write_address, write_vector)

        return read_vector

# Example usage
memory_size = 128
word_size = 64
num_read_heads = 1
num_write_heads = 1

dnc = DNC(memory_size, word_size, num_read_heads, num_write_heads)
inputs = tf.random.normal(shape=(1, 10, 128))  # Example input sequence
outputs = dnc(inputs)
print(outputs)


In [ ]:
import tensorflow as tf
import numpy as np

class NeuralTuringMachine(tf.keras.Model):
    def __init__(self, num_slots, slot_size, num_heads, controller_units):
        super(NeuralTuringMachine, self).__init__()
        self.num_slots = num_slots
        self.slot_size = slot_size
        self.num_heads = num_heads
        self.controller_units = controller_units

        # Memory matrix
        self.memory = tf.Variable(tf.random.normal([num_slots, slot_size]))

        # Controller (LSTM)
        self.controller = tf.keras.layers.LSTM(controller_units, return_state=True)

        # Attention mechanisms
        self.read_head = tf.keras.layers.Dense(slot_size * num_heads)
        self.write_head = tf.keras.layers.Dense(slot_size + 1)

    def call(self, inputs, prev_state):
        # Unpack previous state
        prev_read_weights, prev_controller_state = prev_state

        # Controller input: concatenate input with read vectors
        controller_input = tf.concat([inputs] + prev_read_weights, axis=-1)

        # Controller output
        controller_output, controller_state_h, controller_state_c = self.controller(controller_input, initial_state=prev_controller_state)
        controller_state = [controller_state_h, controller_state_c]

        # Read head
        read_weights = tf.nn.softmax(self.read_head(controller_output))
        read_vectors = tf.matmul(read_weights, self.memory)

        # Write head
        write_weights = tf.nn.sigmoid(self.write_head(controller_output))
        erase_vector = tf.matmul(write_weights[:, :self.slot_size], tf.zeros([1, self.slot_size]))
        add_vector = write_weights[:, self.slot_size:]
        self.memory = self.memory * (1 - erase_vector) + tf.matmul(tf.transpose(add_vector), tf.ones([1, self.slot_size]))

        return read_vectors, [read_weights, controller_state]

# Example usage
ntm = NeuralTuringMachine(num_slots=128, slot_size=20, num_heads=1, controller_units=100)
inputs = tf.random.normal([32, 10])  # Example input sequence
initial_state = [tf.zeros([32, 128]), [tf.zeros([32, 100]), tf.zeros([32, 100])]]  # Initial state
read_vectors, final_state = ntm(inputs, initial_state)

# Inspect output
print("Read vectors shape:", read_vectors.shape)


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to range [0, 1]
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)  # Output layer with 10 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


170498071/170498071 [==============================] - 2s 0us/step
Epoch 1/10
1563/1563 [==============================] - 66s 41ms/step - loss: 1.5428 - accuracy: 0.4337 - val_loss: 1.2664 - val_accuracy: 0.5481
Epoch 2/10
1563/1563 [==============================] - 58s 37ms/step - loss: 1.1645 - accuracy: 0.5854 - val_loss: 1.0895 - val_accuracy: 0.6157
Epoch 3/10
1563/1563 [==============================] - 61s 39ms/step - loss: 1.0176 - accuracy: 0.6422 - val_loss: 0.9944 - val_accuracy: 0.6471
Epoch 4/10
1563/1563 [==============================] - 59s 38ms/step - loss: 0.9171 - accuracy: 0.6783 - val_loss: 0.9323 - val_accuracy: 0.6734
Epoch 5/10
1563/1563 [==============================] - 71s 46ms/step - loss: 0.8426 - accuracy: 0.7069 - val_loss: 0.9191 - val_accuracy: 0.6822
Epoch 6/10
1563/1563 [==============================] - 62s 40ms/step - loss: 0.7828 - accuracy: 0.7268 - val_loss: 0.9554 - val_accuracy: 0.6723
Epoch 7/10
1563/1563 [==============================] - 6

In [13]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the KNN classifier
k = 3  # Number of neighbors
knn = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn.fit(X_train, y_train)

# Predict labels for test set
y_pred = knn.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 1.0


In [16]:
!pip install transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Prepare input text
text = "This is an example sentence for text classification."
inputs = tokenizer(text, return_tensors="pt")

# Perform inference
outputs = model(**inputs)
logits = outputs.logits

# Get predicted class
predicted_class = torch.argmax(logits, dim=1).item()

# Print predicted class
print("Predicted class:", predicted_class)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: 1


In [18]:
!pip install transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Prepare input text
text = "This is an example sentence for text classification."
inputs = tokenizer(text, return_tensors="pt")

# Perform inference
outputs = model(**inputs)
logits = outputs.logits

# Get predicted class
predicted_class = torch.argmax(logits, dim=1).item()

# Print predicted class
print("Predicted class:", predicted_class)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: 1
